<a href="https://colab.research.google.com/github/thangvle/smart_pros/blob/master/trial_Tri_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import tensorflow as tf
import pylab as pl
import numpy as np
import matplotlib
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
# need the following 2 libraries to read csv files
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

# importing EMG data
  # first download the csv file to tensorflow & label it
!wget -O sheetnamedude.csv https://raw.githubusercontent.com/thangvle/smart_pros/master/EMG_label_data_muscle1.csv
  # now we can use that name to read the data (quietly)
data = pd.read_csv("sheetnamedude.csv");

# turn time and voltage to 1D array
data.columns = ["time_stamp", "voltage", "time", "label"]
time = data["time"].values
voltage = data["voltage"].values
labels = data["label"].values
#print("Time loaded");
#print(time);
#print("voltage loaded")
#print(voltage);
#print("label loaded")
#print(labels);

train_data, test_data = train_test_split(data, train_size=0.8)
#print("train_data loaded")
#print(train_data);
#print("test_data loaded")
#print(test_data);


"""
TODO

- Initialize variable for time and voltage as 1D array
- Initialize Weight function
- Create loss function and optimizer (SGD)
- tf.session
"""

def conv_relu(inputs, filters, k_size, stride, padding, scope_name):

    # method that does convolution + relu on inputs

    with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        in_channels = inputs.shape[-1]
        kernel = tf.get_variable('kernel',
                                [k_size, k_size, in_channels, filters],
                                initializer=tf.truncated_normal_initializer())
        biases = tf.get_variable('biases',
                                [filters],
                                initializer=tf.random_normal_initializer())
        conv = tf.nn.conv1d(inputs, kernel, strides=[1, stride, stride, 1], padding=padding)
    return tf.nn.relu(conv + biases, name=scope.name)

def maxpool(inputs, ksize, stride, padding='VALID', scope_name='pool'):
    '''A method that does max pooling on inputs'''
    with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        pool = tf.nn.max_pool(inputs,
                            ksize=[1, ksize, ksize, 1],
                            strides=[1, stride, stride, 1],
                            padding=padding)
    return pool

def fully_connected(inputs, out_dim, scope_name='fc'):
    # A fully connected linear layer on inputs

    with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        in_dim = inputs.shape[-1]
        w = tf.get_variable('weights', [in_dim, out_dim],
                            initializer=tf.truncated_normal_initializer())
        b = tf.get_variable('biases', [out_dim],
                            initializer=tf.constant_initializer(0.0))
        out = tf.matmul(inputs, w) + b
    return out

## let's test out an example for a simple CNN of 2D operation without Tensorflow
import numpy as np
from scipy import signal as sg

I= [[255,   7,  3],
    [212, 240,  4],
    [218, 216, 230],]

g= [[-1, 1]]

print('Without zero padding \n')
print('{0} \n'.format(sg.convolve( I, g, 'valid')))
# The 'valid' argument states that the output consists only of those elements 
# that do not rely on the zero-padding.

print('With zero padding \n')
print(sg.convolve( I, g))



--2019-03-24 03:40:07--  https://raw.githubusercontent.com/thangvle/smart_pros/master/EMG_label_data_muscle1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11285 (11K) [text/plain]
Saving to: ‘sheetnamedude.csv’

sheetnamedude.csv   100%[===================>]  11.02K  --.-KB/s    in 0s      

2019-03-24 03:40:07 (89.5 MB/s) - ‘sheetnamedude.csv’ saved [11285/11285]

Without zero padding 

[[248   4]
 [-28 236]
 [  2 -14]] 

With zero padding 

[[-255  248    4    3]
 [-212  -28  236    4]
 [-218    2  -14  230]]
Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Input 

[[[[ 1.297622  ]
   [-0.66893405]
   [-1.4751151 ]
   [-0.54511726]
   [ 1.3021251 ]
   [ 1.2231294 ]
   [-0.6104273 ]
   [ 0.4424685 ]
   [-1.6794149 ]
   [ 0.21796377]]

  [[ 0.50676143]
   [-0.5456453 ]
   [-0.2884358 ]
   [-0.88433534]
   [ 0.10025802]
   [-1.628593  ]
   [ 0.6514128 ]
   [-0.04617062]
   [-0.58049226]
   [ 1.7791703 ]]

  [[ 1.2507726 ]
   [-0.3233204 ]
   [-0.18588136]
   [-0.52206403]
   [-0.34573564]
   [-1.2743132 ]
   [-1.4109825 ]
   [ 0.58385885]
   [ 1.2101251 ]
   [-0.01924355]]

  [[-0.6895673 ]
   [-0.31945765]
   [ 0.47214866]
   [-1.7567148 ]
   [ 0.16092782]
   [ 0.47786817]
   [ 0.63596725]
   [-0.9022847 ]
   [-1.1658045 ]
   [ 0.14738162]]

  [[ 0.218755  ]
   [-0.3123977 ]
   [-1.742437  ]
   [-0.10360944]
   [-0.7403505 ]
   [ 1.114403  ]
   [ 0.47483638]
   [ 0.07883488]
   [ 1.2749642 ]
   [-0.16868806]]

  [[-2.0168023 ]
   [-1.0260661 ]
   [ 1.4088511 ]
   [ 0.45434013]
   [-1.4978911 ]
   [ 0.6186822 ]
   [ 1.239347  ]
   [ 0.297174

NameError: ignored